# Machine Learning With Spark ML
In this lab assignment, you will complete a project by going through the following steps:
1. Get the data.
2. Discover the data to gain insights.
3. Prepare the data for Machine Learning algorithms.
4. Select a model and train it.
5. Fine-tune your model.
6. Present your solution.

As a dataset, we use the California Housing Prices dataset from the StatLib repository. This dataset was based on data from the 1990 California census. The dataset has the following columns
1. `longitude`: a measure of how far west a house is (a higher value is farther west)
2. `latitude`: a measure of how far north a house is (a higher value is farther north)
3. `housing_,median_age`: median age of a house within a block (a lower number is a newer building)
4. `total_rooms`: total number of rooms within a block
5. `total_bedrooms`: total number of bedrooms within a block
6. `population`: total number of people residing within a block
7. `households`: total number of households, a group of people residing within a home unit, for a block
8. `median_income`: median income for households within a block of houses
9. `median_house_value`: median house value for households within a block
10. `ocean_proximity`: location of the house w.r.t ocean/sea

---
# 1. Get the data
Let's start the lab by loading the dataset. The can find the dataset at `data/housing.csv`. To infer column types automatically, when you are reading the file, you need to set `inferSchema` to true. Moreover enable the `header` option to read the columns' name from the file.

In [0]:
spark.version

Out[2]: '3.0.1'

In [0]:
%scala
val housing = spark.read.format("csv").option("header", "true").option("inferSchema", "true")load("dbfs:/FileStore/shared_uploads/sbsk@kth.se/housing.csv")

housing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 8 more fields]

---
# 2. Discover the data to gain insights
Now it is time to take a look at the data. In this step we are going to take a look at the data a few different ways:
* See the schema and dimension of the dataset
* Look at the data itself
* Statistical summary of the attributes
* Breakdown of the data by the categorical attribute variable
* Find the correlation among different attributes
* Make new attributes by combining existing attributes

## 2.1. Schema and dimension
Print the schema of the dataset

In [0]:
%scala

housing.printSchema()

root
-- longitude: double (nullable = true)
-- latitude: double (nullable = true)
-- housing_median_age: double (nullable = true)
-- total_rooms: double (nullable = true)
-- total_bedrooms: double (nullable = true)
-- population: double (nullable = true)
-- households: double (nullable = true)
-- median_income: double (nullable = true)
-- median_house_value: double (nullable = true)
-- ocean_proximity: string (nullable = true)

Print the number of records in the dataset.

In [0]:
%scala

housing.count

res2: Long = 20640

## 2.2. Look at the data
Print the first five records of the dataset.

In [0]:
%scala

housing.show(5,false)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
-122.23 |37.88 |41.0 |880.0 |129.0 |322.0 |126.0 |8.3252 |452600.0 |NEAR BAY |
-122.22 |37.86 |21.0 |7099.0 |1106.0 |2401.0 |1138.0 |8.3014 |358500.0 |NEAR BAY |
-122.24 |37.85 |52.0 |1467.0 |190.0 |496.0 |177.0 |7.2574 |352100.0 |NEAR BAY |
-122.25 |37.85 |52.0 |1274.0 |235.0 |558.0 |219.0 |5.6431 |341300.0 |NEAR BAY |
-122.25 |37.85 |52.0 |1627.0 |280.0 |565.0 |259.0 |3.8462 |342200.0 |NEAR BAY |
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
only showing top 5 rows

Print the number of records with population more than 10000.

In [0]:
%scala
housing.filter($"population" > 10000).show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
 -121.92| 37.53| 7.0| 28258.0| 3864.0| 12203.0| 3701.0| 8.4045| 451100.0| <1H OCEAN|
 -117.78| 34.03| 8.0| 32054.0| 5290.0| 15507.0| 5050.0| 6.0191| 253900.0| <1H OCEAN|
 -117.87| 34.04| 7.0| 27700.0| 4179.0| 15037.0| 4072.0| 6.6288| 339700.0| <1H OCEAN|
 -117.88| 33.96| 16.0| 19059.0| 3079.0| 10988.0| 3061.0| 5.5469| 265200.0| <1H OCEAN|
 -118.78| 34.16| 9.0| 30405.0| 4093.0| 12873.0| 3931.0| 8.0137| 399200.0| NEAR OCEAN|
 -118.09| 34.68| 4.0| 23386.0| 4171.0| 10493.0| 3671.0| 4.0211| 144000.0| INLAND|
 -118.1| 34.57| 7.0| 20377.0| 4335.0| 11973.0| 3933.0| 3.3086| 138100.0| INLAND|
 -118.46| 34.4| 12.0| 25957.0| 4798.0| 10475.0| 4490.0| 4.542| 195300.0| <1H OCEAN|
 -121.61| 36.69| 19.0| 9899.0| 2617.0| 11272.0| 2528.0| 2.0244| 118500.0| <1H OCEAN|
 -121.68| 36.72| 12.0| 19234.0| 4492.0| 12153.0| 4372.0| 3.2652| 152800.0| <1H OCEAN|
 -121.79| 36.64| 11.0| 32627.0| 6445.0| 28566.0| 6082.0| 2.3087| 118800.0| <1H OCEAN|
 -117.74| 33.89| 4.0| 37937.0| 5471.0| 16122.0| 5189.0| 7.4947| 366300.0| <1H OCEAN|
 -117.12| 33.52| 4.0| 30401.0| 4957.0| 13251.0| 4339.0| 4.5841| 212300.0| <1H OCEAN|
 -121.53| 38.48| 5.0| 27870.0| 5027.0| 11935.0| 4855.0| 4.8811| 212200.0| INLAND|
 -121.4| 38.47| 4.0| 20982.0| 3392.0| 10329.0| 3086.0| 4.3658| 130600.0| INLAND|
 -121.44| 38.43| 3.0| 39320.0| 6210.0| 16305.0| 5358.0| 4.9516| 153700.0| INLAND|
 -117.75| 34.01| 4.0| 22128.0| 3522.0| 10450.0| 3258.0| 6.1287| 289600.0| <1H OCEAN|
 -117.61| 34.1| 9.0| 18956.0| 4095.0| 10323.0| 3832.0| 3.6033| 132600.0| INLAND|
 -116.14| 34.45| 12.0| 8796.0| 1721.0| 11139.0| 1680.0| 2.2612| 137500.0| INLAND|
 -117.42| 33.35| 14.0| 25135.0| 4819.0| 35682.0| 4769.0| 2.5729| 134400.0| <1H OCEAN|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
only showing top 20 rows

## 2.3. Statistical summary
Print a summary of the table statistics for the attributes `housing_median_age`, `total_rooms`, `median_house_value`, and `population`. You can use the `describe` command.

In [0]:
%scala
housing.describe("housing_median_age","total_rooms","median_house_value","population").show()

+-------+------------------+------------------+------------------+------------------+
summary|housing_median_age| total_rooms|median_house_value| population|
+-------+------------------+------------------+------------------+------------------+
 count| 20640| 20640| 20640| 20640|
 mean|28.639486434108527|2635.7630813953488|206855.81690891474|1425.4767441860465|
 stddev| 12.58555761211163|2181.6152515827944|115395.61587441359| 1132.46212176534|
 min| 1.0| 2.0| 14999.0| 3.0|
 max| 52.0| 39320.0| 500001.0| 35682.0|
+-------+------------------+------------------+------------------+------------------+

Print the maximum age (`housing_median_age`), the minimum number of rooms (`total_rooms`), and the average of house values (`median_house_value`).

In [0]:
%scala

import org.apache.spark.sql.functions._

housing.agg(max("housing_median_age")).show()
housing.agg(min("total_rooms")).show()
housing.agg(avg("median_house_value")).show()

+-----------------------+
max(housing_median_age)|
+-----------------------+
 52.0|
+-----------------------+

+----------------+
min(total_rooms)|
+----------------+
 2.0|
+----------------+

+-----------------------+
avg(median_house_value)|
+-----------------------+
 206855.81690891474|
+-----------------------+

import org.apache.spark.sql.functions._

## 2.4. Breakdown the data by categorical data
Print the number of houses in different areas (`ocean_proximity`), and sort them in descending order.

In [0]:
%scala

housing.groupBy("ocean_proximity").count().sort(desc("count")).show()

+---------------+-----+
ocean_proximity|count|
+---------------+-----+
 <1H OCEAN| 9136|
 INLAND| 6551|
 NEAR OCEAN| 2658|
 NEAR BAY| 2290|
 ISLAND| 5|
+---------------+-----+

Print the average value of the houses (`median_house_value`) in different areas (`ocean_proximity`), and call the new column `avg_value` when print it.

In [0]:
%scala

housing.groupBy("ocean_proximity").agg(avg("median_house_value")).withColumnRenamed("avg(median_house_value)", "avg_value").show()

+---------------+------------------+
ocean_proximity| avg_value|
+---------------+------------------+
 ISLAND| 380440.0|
 NEAR OCEAN|249433.97742663656|
 NEAR BAY|259212.31179039303|
 <1H OCEAN|240084.28546409807|
 INLAND|124805.39200122119|
+---------------+------------------+

Rewrite the above question in SQL.

In [0]:
%scala

housing.createOrReplaceTempView("df")
spark.sql("SELECT ocean_proximity, avg(median_house_value) AS avg_value FROM df GROUP BY ocean_proximity").show()

+---------------+------------------+
ocean_proximity| avg_value|
+---------------+------------------+
 ISLAND| 380440.0|
 NEAR OCEAN|249433.97742663656|
 NEAR BAY|259212.31179039303|
 <1H OCEAN|240084.28546409807|
 INLAND|124805.39200122119|
+---------------+------------------+

## 2.5. Correlation among attributes
Print the correlation among the attributes `housing_median_age`, `total_rooms`, `median_house_value`, and `population`. To do so, first you need to put these attributes into one vector. Then, compute the standard correlation coefficient (Pearson) between every pair of attributes in this new vector. To make a vector of these attributes, you can use the `VectorAssembler` Transformer.

In [0]:
%scala

import org.apache.spark.ml.feature.VectorAssembler

val va = new VectorAssembler().setInputCols(Array("housing_median_age","total_rooms","median_house_value","population")).setOutputCol("features")

val housingAttrs = va.transform(housing)

housingAttrs.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+--------------------+
longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity| features|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+--------------------+
 -122.23| 37.88| 41.0| 880.0| 129.0| 322.0| 126.0| 8.3252| 452600.0| NEAR BAY|[41.0,880.0,45260...|
 -122.22| 37.86| 21.0| 7099.0| 1106.0| 2401.0| 1138.0| 8.3014| 358500.0| NEAR BAY|[21.0,7099.0,3585...|
 -122.24| 37.85| 52.0| 1467.0| 190.0| 496.0| 177.0| 7.2574| 352100.0| NEAR BAY|[52.0,1467.0,3521...|
 -122.25| 37.85| 52.0| 1274.0| 235.0| 558.0| 219.0| 5.6431| 341300.0| NEAR BAY|[52.0,1274.0,3413...|
 -122.25| 37.85| 52.0| 1627.0| 280.0| 565.0| 259.0| 3.8462| 342200.0| NEAR BAY|[52.0,1627.0,3422...|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+--------------------+
only showing top 5 rows

import org.apache.spark.ml.feature.VectorAssembler
va: org.apache.spark.ml.feature.VectorAssembler = VectorAssembler: uid=vecAssembler_85fcb9673fb3, handleInvalid=error, numInputCols=4
housingAttrs: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 9 more fields]

In [0]:
%scala

import org.apache.spark.ml.linalg.Matrix
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.sql.Row

val Row(coeff: Matrix) = Correlation.corr(housingAttrs, "features").head

println(s"The standard correlation coefficient:\n ${coeff}")

The standard correlation coefficient:
 1.0 -0.3612622012223152 0.10562341249321029 -0.29624423977353675 
-0.3612622012223152 1.0 0.13415311380656375 0.8571259728659744 
0.10562341249321029 0.13415311380656375 1.0 -0.024649678888894997 
-0.29624423977353675 0.8571259728659744 -0.024649678888894997 1.0 
import org.apache.spark.ml.linalg.Matrix
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.sql.Row
coeff: org.apache.spark.ml.linalg.Matrix =
1.0 -0.3612622012223152 0.10562341249321029 -0.29624423977353675
-0.3612622012223152 1.0 0.13415311380656375 0.8571259728659744
0.10562341249321029 0.13415311380656375 1.0 -0.024649678888894997
-0.29624423977353675 0.8571259728659744 -0.024649678888894997 1.0

## 2.6. Combine and make new attributes
Now, let's try out various attribute combinations. In the given dataset, the total number of rooms in a block is not very useful, if we don't know how many households there are. What we really want is the number of rooms per household. Similarly, the total number of bedrooms by itself is not very useful, and we want to compare it to the number of rooms. And the population per household seems like also an interesting attribute combination to look at. To do so, add the three new columns to the dataset as below. We will call the new dataset the `housingExtra`.
```
rooms_per_household = total_rooms / households
bedrooms_per_room = total_bedrooms / total_rooms
population_per_household = population / households
```

In [0]:
%scala

val housingCol1 = housing.withColumn("rooms_per_household", expr("total_rooms / households"))
val housingCol2 = housingCol1.withColumn("bedrooms_per_room", expr("total_bedrooms / total_rooms"))
val housingExtra = housingCol2.withColumn("population_per_household", expr("population / households"))

housingExtra.select("rooms_per_household", "bedrooms_per_room", "population_per_household").show(5)

+-------------------+-------------------+------------------------+
rooms_per_household| bedrooms_per_room|population_per_household|
+-------------------+-------------------+------------------------+
 6.984126984126984|0.14659090909090908| 2.5555555555555554|
 6.238137082601054|0.15579659106916466| 2.109841827768014|
 8.288135593220339|0.12951601908657123| 2.8022598870056497|
 5.8173515981735155|0.18445839874411302| 2.547945205479452|
 6.281853281853282| 0.1720958819913952| 2.1814671814671813|
+-------------------+-------------------+------------------------+
only showing top 5 rows

housingCol1: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 9 more fields]
housingCol2: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 10 more fields]
housingExtra: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 11 more fields]

---
## 3. Prepare the data for Machine Learning algorithms
Before going through the Machine Learning steps, let's first rename the label column from `median_house_value` to `label`.

In [0]:
%scala

val renamedHousing = housingExtra.withColumnRenamed("median_house_value", "label")

renamedHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 11 more fields]

Now, we want to separate the numerical attributes from the categorical attribute (`ocean_proximity`) and keep their column names in two different lists. Moreover, sice we don't want to apply the same transformations to the predictors (features) and the label, we should remove the label attribute from the list of predictors.

In [0]:
%scala

// label columns
val colLabel = "label"

// categorical columns
val colCat = "ocean_proximity"

// numerical columns
val colNum = renamedHousing.columns.filter(_ != colLabel).filter(_ != colCat)

colLabel: String = label
colCat: String = ocean_proximity
colNum: Array[String] = Array(longitude, latitude, housing_median_age, total_rooms, total_bedrooms, population, households, median_income, rooms_per_household, bedrooms_per_room, population_per_household)

## 3.1. Prepare continuse attributes
### Data cleaning
Most Machine Learning algorithms cannot work with missing features, so we should take care of them. As a first step, let's find the columns with missing values in the numerical attributes. To do so, we can print the number of missing values of each continues attributes, listed in `colNum`.

In [0]:
%scala

for (c <- colNum) 
{
  val count = renamedHousing.filter(renamedHousing(c).isNull || renamedHousing(c) === "" || renamedHousing(c).isNaN).count()
  println("Missing Values in " + c + ": " + count)
}

Missing Values in longitude: 0
Missing Values in latitude: 0
Missing Values in housing_median_age: 0
Missing Values in total_rooms: 0
Missing Values in total_bedrooms: 207
Missing Values in population: 0
Missing Values in households: 0
Missing Values in median_income: 0
Missing Values in rooms_per_household: 0
Missing Values in bedrooms_per_room: 207
Missing Values in population_per_household: 0

As we observerd above, the `total_bedrooms` and `bedrooms_per_room` attributes have some missing values. One way to take care of missing values is to use the `Imputer` Transformer, which completes missing values in a dataset, either using the mean or the median of the columns in which the missing values are located. To use it, you need to create an `Imputer` instance, specifying that you want to replace each attribute's missing values with the "median" of that attribute.

In [0]:
%scala

import org.apache.spark.ml.feature.Imputer

val imputer = new Imputer().setStrategy("median").setInputCols(colNum).setOutputCols(colNum)                                  
val imputedHousing = imputer.fit(renamedHousing).transform(renamedHousing)

imputedHousing.select("total_bedrooms", "bedrooms_per_room").show(5)

+--------------+-------------------+
total_bedrooms| bedrooms_per_room|
+--------------+-------------------+
 129.0|0.14659090909090908|
 1106.0|0.15579659106916466|
 190.0|0.12951601908657123|
 235.0|0.18445839874411302|
 280.0| 0.1720958819913952|
+--------------+-------------------+
only showing top 5 rows

import org.apache.spark.ml.feature.Imputer
imputer: org.apache.spark.ml.feature.Imputer = imputer_f3edbe43dbbb
imputedHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 11 more fields]

### Scaling
One of the most important transformations you need to apply to your data is feature scaling. With few exceptions, Machine Learning algorithms don't perform well when the input numerical attributes have very different scales. This is the case for the housing data: the total number of rooms ranges from about 6 to 39,320, while the median incomes only range from 0 to 15. Note that scaling the label attribues is generally not required.

One way to get all attributes to have the same scale is to use standardization. In standardization, for each value, first it subtracts the mean value (so standardized values always have a zero mean), and then it divides by the variance so that the resulting distribution has unit variance. To do this, we can use the `StandardScaler` Estimator. To use `StandardScaler`, again we need to convert all the numerical attributes into a big vector of features using `VectorAssembler`, and then call `StandardScaler` on that vactor.

In [0]:
%scala

import org.apache.spark.ml.feature.{VectorAssembler, StandardScaler}

val va = new VectorAssembler().setInputCols(colNum).setOutputCol("features")
val featuredHousing = va.transform(imputedHousing)

val scaler = new StandardScaler().setInputCol("features").setOutputCol("scaled_features")
val scaledHousing = scaler.fit(featuredHousing).transform(featuredHousing)

scaledHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+
longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income| label|ocean_proximity|rooms_per_household| bedrooms_per_room|population_per_household| features| scaled_features|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+
 -122.23| 37.88| 41.0| 880.0| 129.0| 322.0| 126.0| 8.3252|452600.0| NEAR BAY| 6.984126984126984|0.14659090909090908| 2.5555555555555554|[-122.23,37.88,41...|[-61.007269596069...|
 -122.22| 37.86| 21.0| 7099.0| 1106.0| 2401.0| 1138.0| 8.3014|358500.0| NEAR BAY| 6.238137082601054|0.15579659106916466| 2.109841827768014|[-122.22,37.86,21...|[-61.002278409814...|
 -122.24| 37.85| 52.0| 1467.0| 190.0| 496.0| 177.0| 7.2574|352100.0| NEAR BAY| 8.288135593220339|0.12951601908657123| 2.8022598870056497|[-122.24,37.85,52...|[-61.012260782324...|
 -122.25| 37.85| 52.0| 1274.0| 235.0| 558.0| 219.0| 5.6431|341300.0| NEAR BAY| 5.8173515981735155|0.18445839874411302| 2.547945205479452|[-122.25,37.85,52...|[-61.017251968579...|
 -122.25| 37.85| 52.0| 1627.0| 280.0| 565.0| 259.0| 3.8462|342200.0| NEAR BAY| 6.281853281853282| 0.1720958819913952| 2.1814671814671813|[-122.25,37.85,52...|[-61.017251968579...|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+
only showing top 5 rows

import org.apache.spark.ml.feature.{VectorAssembler, StandardScaler}
va: org.apache.spark.ml.feature.VectorAssembler = VectorAssembler: uid=vecAssembler_ca6a2be5c527, handleInvalid=error, numInputCols=11
featuredHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 12 more fields]
scaler: org.apache.spark.ml.feature.StandardScaler = stdScal_133d95e36a0d
scaledHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 13 more fields]

## 3.2. Prepare categorical attributes
After imputing and scaling the continuse attributes, we should take care of the categorical attributes. Let's first print the number of distict values of the categirical attribute `ocean_proximity`.

In [0]:
%scala

renamedHousing.groupBy("ocean_proximity").count().show()

+---------------+-----+
ocean_proximity|count|
+---------------+-----+
 ISLAND| 5|
 NEAR OCEAN| 2658|
 NEAR BAY| 2290|
 <1H OCEAN| 9136|
 INLAND| 6551|
+---------------+-----+

### String indexer
Most Machine Learning algorithms prefer to work with numbers. So let's convert the categorical attribute `ocean_proximity` to numbers. To do so, we can use the `StringIndexer` that encodes a string column of labels to a column of label indices. The indices are in [0, numLabels), ordered by label frequencies, so the most frequent label gets index 0.

In [0]:
%scala

import org.apache.spark.ml.feature.StringIndexer

val indexer = new StringIndexer().setInputCol("ocean_proximity").setOutputCol("ocean_proximity_indexed")
val idxHousing = indexer.fit(renamedHousing).transform(renamedHousing)

idxHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+-----------------------+
longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income| label|ocean_proximity|rooms_per_household| bedrooms_per_room|population_per_household|ocean_proximity_indexed|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+-----------------------+
 -122.23| 37.88| 41.0| 880.0| 129.0| 322.0| 126.0| 8.3252|452600.0| NEAR BAY| 6.984126984126984|0.14659090909090908| 2.5555555555555554| 3.0|
 -122.22| 37.86| 21.0| 7099.0| 1106.0| 2401.0| 1138.0| 8.3014|358500.0| NEAR BAY| 6.238137082601054|0.15579659106916466| 2.109841827768014| 3.0|
 -122.24| 37.85| 52.0| 1467.0| 190.0| 496.0| 177.0| 7.2574|352100.0| NEAR BAY| 8.288135593220339|0.12951601908657123| 2.8022598870056497| 3.0|
 -122.25| 37.85| 52.0| 1274.0| 235.0| 558.0| 219.0| 5.6431|341300.0| NEAR BAY| 5.8173515981735155|0.18445839874411302| 2.547945205479452| 3.0|
 -122.25| 37.85| 52.0| 1627.0| 280.0| 565.0| 259.0| 3.8462|342200.0| NEAR BAY| 6.281853281853282| 0.1720958819913952| 2.1814671814671813| 3.0|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+-----------------------+
only showing top 5 rows

import org.apache.spark.ml.feature.StringIndexer
indexer: org.apache.spark.ml.feature.StringIndexer = strIdx_de85fc64a6f3
idxHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 12 more fields]

Now we can use this numerical data in any Machine Learning algorithm. You can look at the mapping that this encoder has learned using the `labels` method: "<1H OCEAN" is mapped to 0, "INLAND" is mapped to 1, etc.

In [0]:
%scala

indexer.fit(renamedHousing).labelsArray

res24: Array[Array[String]] = Array(Array(<1H OCEAN, INLAND, NEAR OCEAN, NEAR BAY, ISLAND))

### One-hot encoding
Now, convert the label indices built in the last step into one-hot vectors. To do this, you can take advantage of the `OneHotEncoderEstimator` Estimator.

In [0]:
%scala

import org.apache.spark.ml.feature.OneHotEncoder

val encoder = new OneHotEncoder().setInputCol("ocean_proximity_indexed").setOutputCol("ocean_proximity_onehotted")
val ohHousing = encoder.fit(idxHousing).transform(idxHousing)

ohHousing.select("ocean_proximity_onehotted").show(5)

+-------------------------+
ocean_proximity_onehotted|
+-------------------------+
 (4,[3],[1.0])|
 (4,[3],[1.0])|
 (4,[3],[1.0])|
 (4,[3],[1.0])|
 (4,[3],[1.0])|
+-------------------------+
only showing top 5 rows

import org.apache.spark.ml.feature.OneHotEncoder
encoder: org.apache.spark.ml.feature.OneHotEncoder = oneHotEncoder_3f851be1f822
ohHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 13 more fields]

---
# 4. Pipeline
As you can see, there are many data transformation steps that need to be executed in the right order. For example, you called the `Imputer`, `VectorAssembler`, and `StandardScaler` from left to right. However, we can use the `Pipeline` class to define a sequence of Transformers/Estimators, and run them in order. A `Pipeline` is an `Estimator`, thus, after a Pipeline's `fit()` method runs, it produces a `PipelineModel`, which is a `Transformer`.

Now, let's create a pipeline called `numPipeline` to call the numerical transformers you built above (`imputer`, `va`, and `scaler`) in the right order from left to right, as well as a pipeline called `catPipeline` to call the categorical transformers (`indexer` and `encoder`). Then, put these two pipelines `numPipeline` and `catPipeline` into one pipeline.

In [0]:
%scala

import org.apache.spark.ml.{Pipeline, PipelineModel, PipelineStage}

val numPipeline = new Pipeline().setStages(Array(imputer, va, scaler))
val catPipeline = new Pipeline().setStages(Array(indexer, encoder))
val pipeline = new Pipeline().setStages(Array(numPipeline, catPipeline))
val newHousing = pipeline.fit(renamedHousing).transform(renamedHousing)

newHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+-----------------------+-------------------------+
longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income| label|ocean_proximity|rooms_per_household| bedrooms_per_room|population_per_household| features| scaled_features|ocean_proximity_indexed|ocean_proximity_onehotted|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+-----------------------+-------------------------+
 -122.23| 37.88| 41.0| 880.0| 129.0| 322.0| 126.0| 8.3252|452600.0| NEAR BAY| 6.984126984126984|0.14659090909090908| 2.5555555555555554|[-122.23,37.88,41...|[-61.007269596069...| 3.0| (4,[3],[1.0])|
 -122.22| 37.86| 21.0| 7099.0| 1106.0| 2401.0| 1138.0| 8.3014|358500.0| NEAR BAY| 6.238137082601054|0.15579659106916466| 2.109841827768014|[-122.22,37.86,21...|[-61.002278409814...| 3.0| (4,[3],[1.0])|
 -122.24| 37.85| 52.0| 1467.0| 190.0| 496.0| 177.0| 7.2574|352100.0| NEAR BAY| 8.288135593220339|0.12951601908657123| 2.8022598870056497|[-122.24,37.85,52...|[-61.012260782324...| 3.0| (4,[3],[1.0])|
 -122.25| 37.85| 52.0| 1274.0| 235.0| 558.0| 219.0| 5.6431|341300.0| NEAR BAY| 5.8173515981735155|0.18445839874411302| 2.547945205479452|[-122.25,37.85,52...|[-61.017251968579...| 3.0| (4,[3],[1.0])|
 -122.25| 37.85| 52.0| 1627.0| 280.0| 565.0| 259.0| 3.8462|342200.0| NEAR BAY| 6.281853281853282| 0.1720958819913952| 2.1814671814671813|[-122.25,37.85,52...|[-61.017251968579...| 3.0| (4,[3],[1.0])|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+-----------------------+-------------------------+
only showing top 5 rows

import org.apache.spark.ml.{Pipeline, PipelineModel, PipelineStage}
numPipeline: org.apache.spark.ml.Pipeline = pipeline_3e8ec4f2d36d
catPipeline: org.apache.spark.ml.Pipeline = pipeline_79f6a79af45e
pipeline: org.apache.spark.ml.Pipeline = pipeline_6d03f24be836
newHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 15 more fields]

Now, use `VectorAssembler` to put all attributes of the final dataset `newHousing` into a big vector, and call the new column `features`.

In [0]:
%scala

val va2 = new VectorAssembler().setInputCols(Array("scaled_features","ocean_proximity_onehotted")).setOutputCol("features_new")
val dataset = va2.transform(newHousing).select("features_new", "label")

dataset.show(5)

+--------------------+--------+
 features_new| label|
+--------------------+--------+
[-61.007269596069...|452600.0|
[-61.002278409814...|358500.0|
[-61.012260782324...|352100.0|
[-61.017251968579...|341300.0|
[-61.017251968579...|342200.0|
+--------------------+--------+
only showing top 5 rows

va2: org.apache.spark.ml.feature.VectorAssembler = VectorAssembler: uid=vecAssembler_b618f78dbc38, handleInvalid=error, numInputCols=2
dataset: org.apache.spark.sql.DataFrame = [features_new: vector, label: double]

In [0]:
%scala

dataset.select("features_new").head

res31: org.apache.spark.sql.Row = [[-61.00726959606955,17.734477624640412,3.2577023016083064,0.40337085073160667,0.30758821710917267,0.2843362208866199,0.3295584480852433,4.382095394195227,2.8228125480951665,2.5405867237343416,0.24605655309533123,0.0,0.0,0.0,1.0]]

---
# 5. Make a model
Here we going to make four different regression models:
* Linear regression model
* Decission tree regression
* Random forest regression
* Gradient-booster forest regression

But, before giving the data to train a Machine Learning model, let's first split the data into training dataset (`trainSet`) with 80% of the whole data, and test dataset (`testSet`) with 20% of it.

In [0]:
%scala

val Array(trainSet, testSet) = dataset.randomSplit(Array(0.8, 0.2))

trainSet: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [features_new: vector, label: double]
testSet: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [features_new: vector, label: double]

## 5.1. Linear regression model
Now, train a Linear Regression model using the `LinearRegression` class. Then, print the coefficients and intercept of the model, as well as the summary of the model over the training set by calling the `summary` method.

In [0]:
%scala

import org.apache.spark.ml.regression.LinearRegression

// train the model
val lr = new LinearRegression().setFeaturesCol("features_new")
val lrModel = lr.fit(trainSet)
val trainingSummary = lrModel.summary

println(s"Coefficients:${lrModel.coefficients} , Intercept: ${lrModel.intercept}")
println(s"RMSE: ${trainingSummary.rootMeanSquaredError}")

Coefficients:[-57522.50979536409,-57907.56491822439,13689.501241114624,3938.5585155072413,1315.1253303828087,-44814.92097940405,45010.67606998363,78470.3861852584,9471.047227451858,17196.40701817732,-5.8189678539577425,-152573.03722514407,-185981.6691386311,-147730.21883379962,-157253.82961629366] , Intercept: -2380752.833915067
RMSE: 67987.8650526237
import org.apache.spark.ml.regression.LinearRegression
lr: org.apache.spark.ml.regression.LinearRegression = linReg_d2efdd675ed2
lrModel: org.apache.spark.ml.regression.LinearRegressionModel = LinearRegressionModel: uid=linReg_d2efdd675ed2, numFeatures=15
trainingSummary: org.apache.spark.ml.regression.LinearRegressionTrainingSummary = org.apache.spark.ml.regression.LinearRegressionTrainingSummary@79c79c4a

Now, use `RegressionEvaluator` to measure the root-mean-square-erroe (RMSE) of the model on the test dataset.

In [0]:
%scala

import org.apache.spark.ml.evaluation.RegressionEvaluator

// make predictions on the test data
val predictions = lrModel.transform(testSet)
predictions.select("prediction", "label", "features_new").show(5)

// select (prediction, true label) and compute test error.
val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+-------+--------------------+
 prediction| label| features_new|
+------------------+-------+--------------------+
103869.38022944843|85800.0|[-62.040445150874...|
 179109.3564704326|79000.0|[-62.025471592109...|
187117.06686240295|76100.0|[-62.015489219599...|
170209.88957228465|69000.0|[-61.985542102068...|
160698.40322666848|70200.0|[-61.980550915813...|
+------------------+-------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 67669.59741678821
import org.apache.spark.ml.evaluation.RegressionEvaluator
predictions: org.apache.spark.sql.DataFrame = [features_new: vector, label: double ... 1 more field]
evaluator: org.apache.spark.ml.evaluation.RegressionEvaluator = RegressionEvaluator: uid=regEval_eb3931bd3ce6, metricName=rmse, throughOrigin=false
rmse: Double = 67669.59741678821

## 5.2. Decision tree regression
Repeat what you have done on Regression Model to build a Decision Tree model. Use the `DecisionTreeRegressor` to make a model and then measure its RMSE on the test dataset.

In [0]:
%scala

import org.apache.spark.ml.regression.DecisionTreeRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

val dt = new DecisionTreeRegressor().setFeaturesCol("features_new")

// train the model
val dtModel = dt.fit(trainSet)

// make predictions on the test data
val predictions = dtModel.transform(testSet)
predictions.select("prediction", "label", "features_new").show(5)

// select (prediction, true label) and compute test error
val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+-------+--------------------+
 prediction| label| features_new|
+------------------+-------+--------------------+
144628.29780361758|85800.0|[-62.040445150874...|
144628.29780361758|79000.0|[-62.025471592109...|
145981.67539267015|76100.0|[-62.015489219599...|
145981.67539267015|69000.0|[-61.985542102068...|
144628.29780361758|70200.0|[-61.980550915813...|
+------------------+-------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 66586.37040342359
import org.apache.spark.ml.regression.DecisionTreeRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator
dt: org.apache.spark.ml.regression.DecisionTreeRegressor = dtr_b8714cc38673
dtModel: org.apache.spark.ml.regression.DecisionTreeRegressionModel = DecisionTreeRegressionModel: uid=dtr_b8714cc38673, depth=5, numNodes=63, numFeatures=15
predictions: org.apache.spark.sql.DataFrame = [features_new: vector, label: double ... 1 more field]
evaluator: org.apache.spark.ml.evaluation.RegressionEvaluator = RegressionEvaluator: uid=regEval_cb3202b271ba, metricName=rmse, throughOrigin=false
rmse: Double = 66586.37040342359

## 5.3. Random forest regression
Let's try the test error on a Random Forest Model. Youcan use the `RandomForestRegressor` to make a Random Forest model.

In [0]:
%scala

import org.apache.spark.ml.regression.RandomForestRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

val rf = new RandomForestRegressor().setFeaturesCol("features_new")

// train the model
val rfModel = rf.fit(trainSet)

// make predictions on the test data
val predictions = rfModel.transform(testSet)
predictions.select("prediction", "label", "features_new").show(5)

// select (prediction, true label) and compute test error
val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+-------+--------------------+
 prediction| label| features_new|
+------------------+-------+--------------------+
 160062.6970698338|85800.0|[-62.040445150874...|
 168277.7845421975|79000.0|[-62.025471592109...|
168953.07447317577|76100.0|[-62.015489219599...|
167811.30062111467|69000.0|[-61.985542102068...|
159776.82077102066|70200.0|[-61.980550915813...|
+------------------+-------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 64588.39707113391
import org.apache.spark.ml.regression.RandomForestRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator
rf: org.apache.spark.ml.regression.RandomForestRegressor = rfr_2c4f9620ce26
rfModel: org.apache.spark.ml.regression.RandomForestRegressionModel = RandomForestRegressionModel: uid=rfr_2c4f9620ce26, numTrees=20, numFeatures=15
predictions: org.apache.spark.sql.DataFrame = [features_new: vector, label: double ... 1 more field]
evaluator: org.apache.spark.ml.evaluation.RegressionEvaluator = RegressionEvaluator: uid=regEval_015b1891c6bf, metricName=rmse, throughOrigin=false
rmse: Double = 64588.39707113391

## 5.4. Gradient-boosted tree regression
Fianlly, we want to build a Gradient-boosted Tree Regression model and test the RMSE of the test data. Use the `GBTRegressor` to build the model.

In [0]:
%scala

import org.apache.spark.ml.regression.GBTRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

val gb = new GBTRegressor().setFeaturesCol("features_new")

// train the model
val gbModel = gb.fit(trainSet)

// make predictions on the test data
val predictions = gbModel.transform(testSet)
predictions.select("prediction", "label", "features_new").show(5)

// select (prediction, true label) and compute test error
val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+-------+--------------------+
 prediction| label| features_new|
+------------------+-------+--------------------+
 81624.25613121284|85800.0|[-62.040445150874...|
 82643.02580017183|79000.0|[-62.025471592109...|
109658.79812621562|76100.0|[-62.015489219599...|
 90251.73604115177|69000.0|[-61.985542102068...|
 79512.09019603758|70200.0|[-61.980550915813...|
+------------------+-------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 55896.484343861885
import org.apache.spark.ml.regression.GBTRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator
gb: org.apache.spark.ml.regression.GBTRegressor = gbtr_7b5b1a4c13a1
gbModel: org.apache.spark.ml.regression.GBTRegressionModel = GBTRegressionModel: uid=gbtr_7b5b1a4c13a1, numTrees=20, numFeatures=15
predictions: org.apache.spark.sql.DataFrame = [features_new: vector, label: double ... 1 more field]
evaluator: org.apache.spark.ml.evaluation.RegressionEvaluator = RegressionEvaluator: uid=regEval_7728c35972cc, metricName=rmse, throughOrigin=false
rmse: Double = 55896.484343861885

---
# 6. Hyperparameter tuning
An important task in Machie Learning is model selection, or using data to find the best model or parameters for a given task. This is also called tuning. Tuning may be done for individual Estimators such as LinearRegression, or for entire Pipelines which include multiple algorithms, featurization, and other steps. Users can tune an entire Pipeline at once, rather than tuning each element in the Pipeline separately. MLlib supports model selection tools, such as `CrossValidator`. These tools require the following items:
* Estimator: algorithm or Pipeline to tune (`setEstimator`)
* Set of ParamMaps: parameters to choose from, sometimes called a "parameter grid" to search over (`setEstimatorParamMaps`)
* Evaluator: metric to measure how well a fitted Model does on held-out test data (`setEvaluator`)

`CrossValidator` begins by splitting the dataset into a set of folds, which are used as separate training and test datasets. For example with `k=3` folds, `CrossValidator` will generate 3 (training, test) dataset pairs, each of which uses 2/3 of the data for training and 1/3 for testing. To evaluate a particular `ParamMap`, `CrossValidator` computes the average evaluation metric for the 3 Models produced by fitting the Estimator on the 3 different (training, test) dataset pairs. After identifying the best `ParamMap`, `CrossValidator` finally re-fits the Estimator using the best ParamMap and the entire dataset.

Below, use the `CrossValidator` to select the best Random Forest model. To do so, you need to define a grid of parameters. Let's say we want to do the search among the different number of trees (1, 5, and 10), and different tree depth (5, 10, and 15).

In [0]:
%scala

import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.tuning.CrossValidator

val paramGrid = new ParamGridBuilder().addGrid(rf.numTrees, Array(1,5,10)).addGrid(rf.maxDepth, Array(5,10,15)).build()

val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
val cv = new CrossValidator().setEstimator(rf).setEvaluator(evaluator).setEstimatorParamMaps(paramGrid).setNumFolds(3)
val cvModel = cv.fit(trainSet)

val predictions = cvModel.transform(testSet)
predictions.select("prediction", "label", "features_new").show(5)

val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+-----------------+-------+--------------------+
 prediction| label| features_new|
+-----------------+-------+--------------------+
89845.55555555555|85800.0|[-62.040445150874...|
86083.47222222222|79000.0|[-62.025471592109...|
 156585.1|76100.0|[-62.015489219599...|
 88895.0|69000.0|[-61.985542102068...|
85367.63888888889|70200.0|[-61.980550915813...|
+-----------------+-------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 51932.676561688204
import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.tuning.CrossValidator
paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	rfr_2c4f9620ce26-maxDepth: 5,
	rfr_2c4f9620ce26-numTrees: 1
}, {
	rfr_2c4f9620ce26-maxDepth: 5,
	rfr_2c4f9620ce26-numTrees: 5
}, {
	rfr_2c4f9620ce26-maxDepth: 5,
	rfr_2c4f9620ce26-numTrees: 10
}, {
	rfr_2c4f9620ce26-maxDepth: 10,
	rfr_2c4f9620ce26-numTrees: 1
}, {
	rfr_2c4f9620ce26-maxDepth: 10,
	rfr_2c4f9620ce26-numTrees: 5
}, {
	rfr_2c4f9620ce26-maxDepth: 10,
	rfr_2c4f9620ce26-numTrees: 10
}, {
	rfr_2c4f9620ce26-maxDepth: 15,
	rfr_2c4f9620ce26-numTrees: 1
}, {
	rfr_2c4f9620ce26-maxDepth: 15,
	rfr_2c4f9620ce26-numTrees: 5
}, {
	rfr_2c4f9620ce26-maxDepth: 15,
	rfr_2c4f9620ce26-numTrees: 10
})
evaluator: org.apache.spark.ml.evaluation.RegressionEvaluator = RegressionEvaluator: uid=regEval_1ddd7b7dd4d4, metricName=rmse, throughOrigin=false
cv: org.apache.spark.ml.tuning.CrossValidator = cv_2a745b3f08ba
cvModel: org.apache.spark.ml.tuning.CrossValidatorModel = CrossValidatorModel: uid=cv_2a745b3f08ba, bestModel=RandomForestRegressionModel: uid=rfr_2c4f9620ce26, numTrees=10, numFeatures=15, numFolds=3
predictions: org.apache.spark.sql.DataFrame = [features_new: vector, label: double ... 1 more field]
rmse: Double = 51932.676561688204

---
# 7. An End-to-End Classification Test
As the last step, you are given a dataset called `data/ccdefault.csv`. The dataset represents default of credit card clients. It has 30,000 cases and 24 different attributes. More details about the dataset is available at `data/ccdefault.txt`. In this task you should make three models, compare their results and conclude the ideal solution. Here are the suggested steps:
1. Load the data.
2. Carry out some exploratory analyses (e.g., how various features and the target variable are distributed).
3. Train a model to predict the target variable (risk of `default`).
  - Employ three different models (logistic regression, decision tree, and random forest).
  - Compare the models' performances (e.g., AUC).
  - Defend your choice of best model (e.g., what are the strength and weaknesses of each of these models?).
4. What more would you do with this data? Anything to help you devise a better solution?

---
## Load the Dataset

1. Loading the dataset into dataframe and printing the schema

In [0]:
%scala

val df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("dbfs:/FileStore/shared_uploads/sbsk@kth.se/ccdefault.csv")

df: org.apache.spark.sql.DataFrame = [ID: int, LIMIT_BAL: int ... 23 more fields]

In [0]:
%scala

df.printSchema()

root
-- ID: integer (nullable = true)
-- LIMIT_BAL: integer (nullable = true)
-- SEX: integer (nullable = true)
-- EDUCATION: integer (nullable = true)
-- MARRIAGE: integer (nullable = true)
-- AGE: integer (nullable = true)
-- PAY_0: integer (nullable = true)
-- PAY_2: integer (nullable = true)
-- PAY_3: integer (nullable = true)
-- PAY_4: integer (nullable = true)
-- PAY_5: integer (nullable = true)
-- PAY_6: integer (nullable = true)
-- BILL_AMT1: integer (nullable = true)
-- BILL_AMT2: integer (nullable = true)
-- BILL_AMT3: integer (nullable = true)
-- BILL_AMT4: integer (nullable = true)
-- BILL_AMT5: integer (nullable = true)
-- BILL_AMT6: integer (nullable = true)
-- PAY_AMT1: integer (nullable = true)
-- PAY_AMT2: integer (nullable = true)
-- PAY_AMT3: integer (nullable = true)
-- PAY_AMT4: integer (nullable = true)
-- PAY_AMT5: integer (nullable = true)
-- PAY_AMT6: integer (nullable = true)
-- DEFAULT: integer (nullable = true)

2.Printing the number of records in the dataset

In [0]:
%scala

df.count()

res51: Long = 30000

In [0]:
%scala

df.show(5)

+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+
 ID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|DEFAULT|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+
 1| 20000| 2| 2| 1| 24| 2| 2| -1| -1| -2| -2| 3913| 3102| 689| 0| 0| 0| 0| 689| 0| 0| 0| 0| 1|
 2| 120000| 2| 2| 2| 26| -1| 2| 0| 0| 0| 2| 2682| 1725| 2682| 3272| 3455| 3261| 0| 1000| 1000| 1000| 0| 2000| 1|
 3| 90000| 2| 2| 2| 34| 0| 0| 0| 0| 0| 0| 29239| 14027| 13559| 14331| 14948| 15549| 1518| 1500| 1000| 1000| 1000| 5000| 0|
 4| 50000| 2| 2| 1| 37| 0| 0| 0| 0| 0| 0| 46990| 48233| 49291| 28314| 28959| 29547| 2000| 2019| 1200| 1100| 1069| 1000| 0|
 5| 50000| 1| 2| 1| 57| -1| 0| -1| 0| 0| 0| 8617| 5670| 35835| 20940| 19146| 19131| 2000| 36681| 10000| 9000| 689| 679| 0|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+
only showing top 5 rows

---
## Carry out exploratory analyses and visualize distributions

---
1. Distribution of sex

In [0]:
%scala

df.groupBy("SEX").count().sort(asc("SEX")).show

+---+-----+
SEX|count|
+---+-----+
 1|11888|
 2|18112|
+---+-----+

2.Distribution of education

In [0]:
%scala

df.groupBy("EDUCATION").count().sort(asc("EDUCATION")).show

+---------+-----+
EDUCATION|count|
+---------+-----+
 0| 14|
 1|10585|
 2|14030|
 3| 4917|
 4| 123|
 5| 280|
 6| 51|
+---------+-----+

3.Distribution of relationship status

In [0]:
%scala

df.groupBy("MARRIAGE").count().sort(asc("MARRIAGE")).show

+--------+-----+
MARRIAGE|count|
+--------+-----+
 0| 54|
 1|13659|
 2|15964|
 3| 323|
+--------+-----+

4.Distribution of age

In [0]:
%scala

df.groupBy("AGE").count().sort(asc("AGE")).show

+---+-----+
AGE|count|
+---+-----+
 21| 67|
 22| 560|
 23| 931|
 24| 1127|
 25| 1186|
 26| 1256|
 27| 1477|
 28| 1409|
 29| 1605|
 30| 1395|
 31| 1217|
 32| 1158|
 33| 1146|
 34| 1162|
 35| 1113|
 36| 1108|
 37| 1041|
 38| 944|
 39| 954|
 40| 870|
+---+-----+
only showing top 20 rows

5.Distribution of given credit

In [0]:
%scala

df.groupBy("LIMIT_BAL").count().show

+---------+-----+
LIMIT_BAL|count|
+---------+-----+
 450000| 161|
 610000| 11|
 350000| 231|
 410000| 78|
 500000| 722|
 260000| 521|
 130000| 729|
 210000| 730|
 270000| 238|
 220000| 469|
 340000| 217|
 16000| 2|
 730000| 2|
 330000| 173|
 740000| 2|
 780000| 2|
 490000| 64|
 327680| 1|
 100000| 1048|
 240000| 619|
+---------+-----+
only showing top 20 rows

6.Distribution of all important categorical features in a single array to do analyses

In [0]:
%scala

val criticalfeatures = Array("DEFAULT", "SEX", "EDUCATION", "MARRIAGE", "PAY_0", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6")

for (catfeat <- criticalfeatures) 
{
    df.groupBy(catfeat).count().sort(desc("count")).show()
}

+-------+-----+
DEFAULT|count|
+-------+-----+
 0|23364|
 1| 6636|
+-------+-----+

+---+-----+
SEX|count|
+---+-----+
 2|18112|
 1|11888|
+---+-----+

+---------+-----+
EDUCATION|count|
+---------+-----+
 2|14030|
 1|10585|
 3| 4917|
 5| 280|
 4| 123|
 6| 51|
 0| 14|
+---------+-----+

+--------+-----+
MARRIAGE|count|
+--------+-----+
 2|15964|
 1|13659|
 3| 323|
 0| 54|
+--------+-----+

+-----+-----+
PAY_0|count|
+-----+-----+
 0|14737|
 -1| 5686|
 1| 3688|
 -2| 2759|
 2| 2667|
 3| 322|
 4| 76|
 5| 26|
 8| 19|
 6| 11|
 7| 9|
+-----+-----+

+-----+-----+
PAY_2|count|
+-----+-----+
 0|15730|
 -1| 6050|
 2| 3927|
 -2| 3782|
 3| 326|
 4| 99|
 1| 28|
 5| 25|
 7| 20|
 6| 12|
 8| 1|
+-----+-----+

+-----+-----+
PAY_3|count|
+-----+-----+
 0|15764|
 -1| 5938|
 -2| 4085|
 2| 3819|
 3| 240|
 4| 76|
 7| 27|
 6| 23|
 5| 21|
 1| 4|
 8| 3|
+-----+-----+

+-----+-----+
PAY_4|count|
+-----+-----+
 0|16455|
 -1| 5687|
 -2| 4348|
 2| 3159|
 3| 180|
 4| 69|
 7| 58|
 5| 35|
 6| 5|
 1| 2|
 8| 2|
+-----+-----+

+-----+-----+
PAY_5|count|
+-----+-----+
 0|16947|
 -1| 5539|
 -2| 4546|
 2| 2626|
 3| 178|
 4| 84|
 7| 58|
 5| 17|
 6| 4|
 8| 1|
+-----+-----+

+-----+-----+
PAY_6|count|
+-----+-----+
 0|16286|
 -1| 5740|
 -2| 4895|
 2| 2766|
 3| 184|
 4| 49|
 7| 46|
 6| 19|
 5| 13|
 8| 2|
+-----+-----+

criticalfeatures: Array[String] = Array(DEFAULT, SEX, EDUCATION, MARRIAGE, PAY_0, PAY_2, PAY_3, PAY_4, PAY_5, PAY_6)

From the tables we can conclude the following results:
1. There are more female instances than male instances as per the distribution of sex.
2. There are more number of participants from university and school than high school with almost half of the participants being single and also another close to half being married with very few having unknown marital status.
3. 'PAY_X' denotes the payment status for the participants. 'PAY_0' denotes the payment status on September where most participants have been paid on time with very few delayed and for the rest of the 7 months also, it seems like the majority have been paid without delay.
4. Since the total number of positive instances (label = 1.0) is lower than the total number of negative instances (label = 0.0), there exists a class imbalance.

## Preparing the data
Deleting the id variable and printing schema again

In [0]:
%scala

val dfnew = df.withColumnRenamed("DEFAULT", "label").drop("ID")
dfnew.printSchema()

root
-- LIMIT_BAL: integer (nullable = true)
-- SEX: integer (nullable = true)
-- EDUCATION: integer (nullable = true)
-- MARRIAGE: integer (nullable = true)
-- AGE: integer (nullable = true)
-- PAY_0: integer (nullable = true)
-- PAY_2: integer (nullable = true)
-- PAY_3: integer (nullable = true)
-- PAY_4: integer (nullable = true)
-- PAY_5: integer (nullable = true)
-- PAY_6: integer (nullable = true)
-- BILL_AMT1: integer (nullable = true)
-- BILL_AMT2: integer (nullable = true)
-- BILL_AMT3: integer (nullable = true)
-- BILL_AMT4: integer (nullable = true)
-- BILL_AMT5: integer (nullable = true)
-- BILL_AMT6: integer (nullable = true)
-- PAY_AMT1: integer (nullable = true)
-- PAY_AMT2: integer (nullable = true)
-- PAY_AMT3: integer (nullable = true)
-- PAY_AMT4: integer (nullable = true)
-- PAY_AMT5: integer (nullable = true)
-- PAY_AMT6: integer (nullable = true)
-- label: integer (nullable = true)

dfnew: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 22 more fields]

Divide columns into label, numerical and categorical

In [0]:
%scala

//Label columns
val colLabel = "label"

//Feature columns
val colFeat = dfnew.columns.filter(_ != colLabel)

//Categorical columns
val colCat = Array("SEX", "EDUCATION", "MARRIAGE", "PAY_0", "PAY_2", "PAY_3",
                   "PAY_4", "PAY_5", "PAY_6")

// Numerical columns
val colNum = Array("LIMIT_BAL", "AGE", "BILL_AMT1", "BILL_AMT2", "BILL_AMT3",
                   "BILL_AMT4", "BILL_AMT5", "BILL_AMT6", "PAY_AMT1", "PAY_AMT2",
                   "PAY_AMT3", "PAY_AMT4", "PAY_AMT5", "PAY_AMT6")

colLabel: String = label
colFeat: Array[String] = Array(LIMIT_BAL, SEX, EDUCATION, MARRIAGE, AGE, PAY_0, PAY_2, PAY_3, PAY_4, PAY_5, PAY_6, BILL_AMT1, BILL_AMT2, BILL_AMT3, BILL_AMT4, BILL_AMT5, BILL_AMT6, PAY_AMT1, PAY_AMT2, PAY_AMT3, PAY_AMT4, PAY_AMT5, PAY_AMT6)
colCat: Array[String] = Array(SEX, EDUCATION, MARRIAGE, PAY_0, PAY_2, PAY_3, PAY_4, PAY_5, PAY_6)
colNum: Array[String] = Array(LIMIT_BAL, AGE, BILL_AMT1, BILL_AMT2, BILL_AMT3, BILL_AMT4, BILL_AMT5, BILL_AMT6, PAY_AMT1, PAY_AMT2, PAY_AMT3, PAY_AMT4, PAY_AMT5, PAY_AMT6)

Prepping categorical features for one-hot encoding

In [0]:
%scala

val df01 = dfnew.na.replace("PAY_0", Map(-1 -> 10))
val df02 = df01.na.replace("PAY_0", Map(-2 -> 11))
val df21 = df02.na.replace("PAY_2", Map(-1 -> 10))
val df22 = df21.na.replace("PAY_2", Map(-2 -> 11))
val df31 = df22.na.replace("PAY_3", Map(-1 -> 10))
val df32 = df31.na.replace("PAY_3", Map(-2 -> 11))
val df41 = df32.na.replace("PAY_4", Map(-1 -> 10))
val df42 = df41.na.replace("PAY_4", Map(-2 -> 11))
val df51 = df42.na.replace("PAY_5", Map(-1 -> 10))
val df52 = df51.na.replace("PAY_5", Map(-2 -> 11))
val df61 = df52.na.replace("PAY_6", Map(-1 -> 10))
val dfUpdated = df61.na.replace("PAY_6", Map(-2 -> 11))

df01: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 22 more fields]
df02: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 22 more fields]
df21: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 22 more fields]
df22: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 22 more fields]
df31: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 22 more fields]
df32: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 22 more fields]
df41: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 22 more fields]
df42: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 22 more fields]
df51: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 22 more fields]
df52: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 22 more fields]
df61: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 22 more fields]
dfUpdated: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 22 more fields]

Checking for missing values

In [0]:
%scala

for (c <- colFeat) 
{
    val missingCount = dfUpdated.filter(dfnew(c).isNull || 
            dfUpdated(c) === "" || dfUpdated(c).isNaN).count() 
    printf("Missing values for " + c + " : " + missingCount + "\n")
}

Missing values for LIMIT_BAL : 0
Missing values for SEX : 0
Missing values for EDUCATION : 0
Missing values for MARRIAGE : 0
Missing values for AGE : 0
Missing values for PAY_0 : 0
Missing values for PAY_2 : 0
Missing values for PAY_3 : 0
Missing values for PAY_4 : 0
Missing values for PAY_5 : 0
Missing values for PAY_6 : 0
Missing values for BILL_AMT1 : 0
Missing values for BILL_AMT2 : 0
Missing values for BILL_AMT3 : 0
Missing values for BILL_AMT4 : 0
Missing values for BILL_AMT5 : 0
Missing values for BILL_AMT6 : 0
Missing values for PAY_AMT1 : 0
Missing values for PAY_AMT2 : 0
Missing values for PAY_AMT3 : 0
Missing values for PAY_AMT4 : 0
Missing values for PAY_AMT5 : 0
Missing values for PAY_AMT6 : 0

Scaling the numerical variables

In [0]:
%scala

val va = new VectorAssembler().setInputCols(colNum).setOutputCol("Features_to_Scale")
val dfToScale = va.transform(dfUpdated)

val scaler = new StandardScaler().setInputCol("Features_to_Scale").setOutputCol("Features_Scaled")
val dfScaled = scaler.fit(dfToScale).transform(dfToScale)

dfScaled.select("Features_Scaled").show(5)

+--------------------+
 Features_Scaled|
+--------------------+
(14,[0,1,2,3,4,9]...|
[0.92487215993365...|
[0.69365411995024...|
[0.38536339997235...|
[0.38536339997235...|
+--------------------+
only showing top 5 rows

va: org.apache.spark.ml.feature.VectorAssembler = VectorAssembler: uid=vecAssembler_ac18aea44660, handleInvalid=error, numInputCols=14
dfToScale: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 23 more fields]
scaler: org.apache.spark.ml.feature.StandardScaler = stdScal_7143f4762abd
dfScaled: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 24 more fields]

In [0]:
%scala
dfScaled.select("Features_Scaled").show(1, false)

+--------------------------------------------------------------------------------------------------------------------------------------------+
Features_Scaled |
+--------------------------------------------------------------------------------------------------------------------------------------------+
(14,[0,1,2,3,4,9],[0.15414535998894324,2.603628744963987,0.053139869207970765,0.0435834725779124,0.009935199510232218,0.029903384202815683])|
+--------------------------------------------------------------------------------------------------------------------------------------------+
only showing top 1 row

## One hot encoding the categorical features

In [0]:
%scala

val encoder = new OneHotEncoder().setInputCols(colCat).setOutputCols(colCat map (name => s"${name}_onehotted"))
val dfOhe = encoder.fit(dfScaled).transform(dfScaled)

dfOhe.select("PAY_3_onehotted").show(10)

+---------------+
PAY_3_onehotted|
+---------------+
(11,[10],[1.0])|
 (11,[0],[1.0])|
 (11,[0],[1.0])|
 (11,[0],[1.0])|
(11,[10],[1.0])|
 (11,[0],[1.0])|
 (11,[0],[1.0])|
(11,[10],[1.0])|
 (11,[2],[1.0])|
 (11,[],[])|
+---------------+
only showing top 10 rows

encoder: org.apache.spark.ml.feature.OneHotEncoder = oneHotEncoder_d42d646d42de
dfOhe: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 33 more fields]

## Creating the final dataframe

In [0]:
%scala

val va = new VectorAssembler()
    .setInputCols(Array("Features_Scaled", "MARRIAGE_onehotted", "EDUCATION_onehotted", "SEX_onehotted",
                        "PAY_0_onehotted", "PAY_2_onehotted", "PAY_3_onehotted", "PAY_4_onehotted",
                        "PAY_5_onehotted", "PAY_6_onehotted"))
    .setOutputCol("features")

val dataset = va.transform(dfOhe).select("features", "label")

dataset.show(5)

+--------------------+-----+
 features|label|
+--------------------+-----+
(91,[0,1,2,3,4,9,...| 1|
(91,[0,1,2,3,4,5,...| 1|
(91,[0,1,2,3,4,5,...| 0|
(91,[0,1,2,3,4,5,...| 0|
(91,[0,1,2,3,4,5,...| 0|
+--------------------+-----+
only showing top 5 rows

va: org.apache.spark.ml.feature.VectorAssembler = VectorAssembler: uid=vecAssembler_3d3d9e20b3fe, handleInvalid=error, numInputCols=10
dataset: org.apache.spark.sql.DataFrame = [features: vector, label: int]

In [0]:
%scala
dataset.show(1, false)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
features |label|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
(91,[0,1,2,3,4,9,15,19,27,38,57,68],[0.15414535998894324,2.603628744963987,0.053139869207970765,0.0435834725779124,0.009935199510232218,0.029903384202815683,1.0,1.0,1.0,1.0,1.0,1.0])|1 |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
only showing top 1 row

## Splitting the final dataset into training set of 80% and test set of 20%

In [0]:
%scala

val Array(trainSet, testSet) = dataset.randomSplit(Array(0.8, 0.2))

trainSet.show(5)
testSet.show(5)

+--------------------+-----+
 features|label|
+--------------------+-----+
(91,[0,1,2,3,4,5,...| 0|
(91,[0,1,2,3,4,5,...| 0|
(91,[0,1,2,3,4,5,...| 0|
(91,[0,1,2,3,4,5,...| 1|
(91,[0,1,2,3,4,5,...| 0|
+--------------------+-----+
only showing top 5 rows

+--------------------+-----+
 features|label|
+--------------------+-----+
(91,[0,1,2,3,4,5,...| 0|
(91,[0,1,2,3,4,5,...| 0|
(91,[0,1,2,3,4,5,...| 0|
(91,[0,1,2,3,4,5,...| 0|
(91,[0,1,2,3,4,5,...| 0|
+--------------------+-----+
only showing top 5 rows

trainSet: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [features: vector, label: int]
testSet: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [features: vector, label: int]

Helper functions to compute Accuracy, Recall, Precision and F1-score

In [0]:
%scala
import org.apache.spark.sql.{DataFrame, Dataset, Row}
def compute_metrics(predictions: DataFrame, str: String)={
    
    val lp = predictions.select( "label", "prediction")
    val count_total = predictions.count()
    val correct = lp.filter($"label" === $"prediction").count()
    val wrong = lp.filter(not($"label" === $"prediction")).count()
    val TP = lp.filter($"prediction" === 1.0).filter($"label" === $"prediction").count()
    val TN = lp.filter($"prediction" === 0.0).filter($"label" === $"prediction").count()
    val FN = lp.filter($"prediction" === 0.0).filter(not($"label" === $"prediction")).count()
    val FP = lp.filter($"prediction" === 1.0).filter(not($"label" === $"prediction")).count()

    print("\nFor the ")
    print(str)
    print(" classifier, the following evaluation metrics were computed: ")
    
    val accuracy = correct.toDouble/count_total.toDouble
    print("\nThe accuracy is: ")
    print(accuracy)

    val precision = TP.toDouble / (TP+FP).toDouble
    print("\nThe precision is: ")
    print(precision)

    val recall = TP.toDouble / (TP+FN).toDouble
    print("\nThe recall is: ")
    print(recall)

    val f1_score = (2*precision*recall).toDouble / (precision+recall).toDouble
    print("\nThe F1 score is: ")
    print(f1_score)
}

import org.apache.spark.sql.{DataFrame, Dataset, Row}
compute_metrics: (predictions: org.apache.spark.sql.DataFrame, str: String)Unit

## Train 3 classification models

## 1. Logistic Regression Model

In [0]:
%scala

import org.apache.spark.ml.classification.{DecisionTreeClassifier, RandomForestClassifier, LogisticRegression}
import org.apache.spark.ml.evaluation.{RegressionEvaluator, BinaryClassificationEvaluator}
import org.apache.spark.ml.param.{ParamMap, Param, Params}

// Initiating the LR model
val lrModel = new LogisticRegression().setLabelCol("label").setFeaturesCol("features").setMaxIter(50)

// Defining the hyper-parameter grid
val paramGrid = new ParamGridBuilder().addGrid(lrModel.regParam, Array(0.1, 0.05, 0.01, 0)).addGrid(lrModel.elasticNetParam, Array(0.1, 0.05, 0.01, 0)).build()

// The BinaryClassificationEvaluator evaluates on the Area Under the ROC-curve (AUC) metric
val evaluator = new BinaryClassificationEvaluator()
    
// Performing cross-validation for model selection
val cv = new CrossValidator().setEstimator(lrModel).setEstimatorParamMaps(paramGrid).setEvaluator(evaluator).setNumFolds(5)

// Training the model
val cvModel = cv.fit(trainSet)

// Making predictions on test data
val predictions = cvModel.transform(testSet)

print("The attributes of the best logistric regression model are:\n")
print(cvModel.bestModel.extractParamMap())

print("\n and the AUC metric for that model is: ")
print(evaluator.evaluate(predictions))

// Computing metrics for the classifier
compute_metrics(predictions, "Logistic Regression")

The attributes of the best logistric regression model are:
{
	logreg_2fd0d527c5b6-aggregationDepth: 2,
	logreg_2fd0d527c5b6-elasticNetParam: 0.05,
	logreg_2fd0d527c5b6-family: auto,
	logreg_2fd0d527c5b6-featuresCol: features,
	logreg_2fd0d527c5b6-fitIntercept: true,
	logreg_2fd0d527c5b6-labelCol: label,
	logreg_2fd0d527c5b6-maxIter: 50,
	logreg_2fd0d527c5b6-predictionCol: prediction,
	logreg_2fd0d527c5b6-probabilityCol: probability,
	logreg_2fd0d527c5b6-rawPredictionCol: rawPrediction,
	logreg_2fd0d527c5b6-regParam: 0.01,
	logreg_2fd0d527c5b6-standardization: true,
	logreg_2fd0d527c5b6-threshold: 0.5,
	logreg_2fd0d527c5b6-tol: 1.0E-6
}
 and the AUC metric for that model is: 0.7664422258996962
For the Logistic Regression classifier, the following evaluation metrics were computed: 
The accuracy is: 0.8148648648648649
The precision is: 0.670487106017192
The recall is: 0.35082458770614694
The F1 score is: 0.46062992125984253import org.apache.spark.ml.classification.{DecisionTreeClassifier, RandomForestClassifier, LogisticRegression}
import org.apache.spark.ml.evaluation.{RegressionEvaluator, BinaryClassificationEvaluator}
import org.apache.spark.ml.param.{ParamMap, Param, Params}
lrModel: org.apache.spark.ml.classification.LogisticRegression = logreg_2fd0d527c5b6
paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	logreg_2fd0d527c5b6-elasticNetParam: 0.1,
	logreg_2fd0d527c5b6-regParam: 0.1
}, {
	logreg_2fd0d527c5b6-elasticNetParam: 0.05,
	logreg_2fd0d527c5b6-regParam: 0.1
}, {
	logreg_2fd0d527c5b6-elasticNetParam: 0.01,
	logreg_2fd0d527c5b6-regParam: 0.1
}, {
	logreg_2fd0d527c5b6-elasticNetParam: 0.0,
	logreg_2fd0d527c5b6-regParam: 0.1
}, {
	logreg_2fd0d527c5b6-elasticNetParam: 0.1,
	logreg_2fd0d527c5b6-regParam: 0.05
}, {
	logreg_2fd0d527c5b6-elasticNetParam: 0.05,
	logreg_2fd0d527c5b6-regParam: 0.05
}, {
	logreg_2fd0d527c5b6-elasticNetParam: 0.01,
	logreg_2fd0d527c5b6-regParam: 0.05
}, {
	logreg_2fd0d527c5b6-elasticNetParam: 0.0,
	logreg_2fd0d527c5b6-regParam: 0.05
}, {
	logreg_2fd0d527c5b6-elasticNetParam: 0.1,
	logreg_2fd0d527c5b6-regParam: 0.01
}, {
	logreg_2fd0d527c5b6-elasticNetParam: 0.05,
	logreg_2fd0d527c5b6-regParam: 0.01
}, {
	logreg_2fd0d527c5b6-elasticNetParam: 0.01,
	logreg_2fd0d527c5b6-regParam: 0.01
}, {
	logreg_2fd0d527c5b6-elasticNetParam: 0.0,
	logreg_2fd0d527c5b6-regParam: 0.01
}, {
	logreg_2fd0d527c5b6-elasticNetParam: 0.1,
	logreg_2fd0d527c5b6-regParam: 0.0
}, {
	logreg_2fd0d527c5b6-elasticNetParam: 0.05,
	logreg_2fd0d527c5b6-regParam: 0.0
}, {
	logreg_2fd0d527c5b6-elasticNetParam: 0.01,
	logreg_2fd0d527c5b6-regParam: 0.0
}, {
	logreg_2fd0d527c5b6-elasticNetParam: 0.0,
	logreg_2fd0d527c5b6-regParam: 0.0
})
evaluator: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = BinaryClassificationEvaluator: uid=binEval_c9909745d6d9, metricName=areaUnderROC, numBins=1000
cv: org.apache.spark.ml.tuning.CrossValidator = cv_e80a9a1a96ef
cvModel: org.apache.spark.ml.tuning.CrossValidatorModel = CrossValidatorModel: uid=cv_e80a9a1a96ef, bestModel=LogisticRegressionModel: uid=logreg_2fd0d527c5b6, numClasses=2, numFeatures=91, numFolds=5
predictions: org.apache.spark.sql.DataFrame = [features: vector, label: int ... 3 more fields]

## 2. Decision Tree Classification Model

In [0]:
%scala

// Initiating the DT model
val dtModel = new DecisionTreeClassifier().setLabelCol("label").setFeaturesCol("features")

// Defining the hyper-parameter grid
val paramGrid = new ParamGridBuilder()
    .addGrid(dtModel.impurity, Array("entropy", "gini"))
    .addGrid(dtModel.maxDepth, Array(4, 5, 6, 7, 8, 9, 10))
    .addGrid(dtModel.minInstancesPerNode, Array(1, 2))//, 3))
    .build()

// Evaluating AUC using BCE
val evaluator = new BinaryClassificationEvaluator()

// Performing cross-validation for model selection
val cv = new CrossValidator().setEstimator(dtModel).setEstimatorParamMaps(paramGrid).setEvaluator(evaluator).setNumFolds(5)

// Training the model
val cvModel = cv.fit(trainSet)

// Making predictions on test data
val predictions = cvModel.transform(testSet)

print("The attributes of the best decision tree classification model are:\n")
print(cvModel.bestModel.extractParamMap())

print("\n and the AUC metric for that model is: ")
print(evaluator.evaluate(predictions))

// Computing other metrics for the classifier
compute_metrics(predictions, "Decision Tree")

The attributes of the best decision tree classification model are:
{
	dtc_53326fbc1c6a-cacheNodeIds: false,
	dtc_53326fbc1c6a-checkpointInterval: 10,
	dtc_53326fbc1c6a-featuresCol: features,
	dtc_53326fbc1c6a-impurity: entropy,
	dtc_53326fbc1c6a-labelCol: label,
	dtc_53326fbc1c6a-leafCol: ,
	dtc_53326fbc1c6a-maxBins: 32,
	dtc_53326fbc1c6a-maxDepth: 10,
	dtc_53326fbc1c6a-maxMemoryInMB: 256,
	dtc_53326fbc1c6a-minInfoGain: 0.0,
	dtc_53326fbc1c6a-minInstancesPerNode: 1,
	dtc_53326fbc1c6a-minWeightFractionPerNode: 0.0,
	dtc_53326fbc1c6a-predictionCol: prediction,
	dtc_53326fbc1c6a-probabilityCol: probability,
	dtc_53326fbc1c6a-rawPredictionCol: rawPrediction,
	dtc_53326fbc1c6a-seed: 159147643
}
 and the AUC metric for that model is: 0.47932507252697254
For the Decision Tree classifier, the following evaluation metrics were computed: 
The accuracy is: 0.8072635135135136
The precision is: 0.6446776611694153
The recall is: 0.32233883058470764
The F1 score is: 0.42978510744627685dtModel: org.apache.spark.ml.classification.DecisionTreeClassifier = dtc_53326fbc1c6a
paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	dtc_53326fbc1c6a-impurity: entropy,
	dtc_53326fbc1c6a-maxDepth: 4,
	dtc_53326fbc1c6a-minInstancesPerNode: 1
}, {
	dtc_53326fbc1c6a-impurity: gini,
	dtc_53326fbc1c6a-maxDepth: 4,
	dtc_53326fbc1c6a-minInstancesPerNode: 1
}, {
	dtc_53326fbc1c6a-impurity: entropy,
	dtc_53326fbc1c6a-maxDepth: 5,
	dtc_53326fbc1c6a-minInstancesPerNode: 1
}, {
	dtc_53326fbc1c6a-impurity: gini,
	dtc_53326fbc1c6a-maxDepth: 5,
	dtc_53326fbc1c6a-minInstancesPerNode: 1
}, {
	dtc_53326fbc1c6a-impurity: entropy,
	dtc_53326fbc1c6a-maxDepth: 6,
	dtc_53326fbc1c6a-minInstancesPerNode: 1
}, {
	dtc_53326fbc1c6a-impurity: gini,
	dtc_53326fbc1c6a-maxDepth: 6,
	dtc_53326fbc1c6a-minInstancesPerNode: 1
}, {
	dtc_53326fbc1c6a-impurity: entropy,
	dtc_53326fbc1c6a-maxDepth: 7,
	dtc_53326fbc1c6a-minInstancesPerNode: 1
}, {
	dtc_53326fbc1c6a-impurity: gini,
	dtc_53326fbc1c6a-maxDepth: 7,
	dtc_53326fbc1c6a-minInstancesPerNode: 1
}, {
	dtc_53326fbc1c6a-impurity: entropy,
	dtc_53326fbc1c6a-maxDepth: 8,
	dtc_53326fbc1c6a-minInstancesPerNode: 1
}, {
	dtc_53326fbc1c6a-impurity: gini,
	dtc_53326fbc1c6a-maxDepth: 8,
	dtc_53326fbc1c6a-minInstancesPerNode: 1
}, {
	dtc_53326fbc1c6a-impurity: entropy,
	dtc_53326fbc1c6a-maxDepth: 9,
	dtc_53326fbc1c6a-minInstancesPerNode: 1
}, {
	dtc_53326fbc1c6a-impurity: gini,
	dtc_53326fbc1c6a-maxDepth: 9,
	dtc_53326fbc1c6a-minInstancesPerNode: 1
}, {
	dtc_53326fbc1c6a-impurity: entropy,
	dtc_53326fbc1c6a-maxDepth: 10,
	dtc_53326fbc1c6a-minInstancesPerNode: 1
}, {
	dtc_53326fbc1c6a-impurity: gini,
	dtc_53326fbc1c6a-maxDepth: 10,
	dtc_53326fbc1c6a-minInstancesPerNode: 1
}, {
	dtc_53326fbc1c6a-impurity: entropy,
	dtc_53326fbc1c6a-maxDepth: 4,
	dtc_53326fbc1c6a-minInstancesPerNode: 2
}, {
	dtc_53326fbc1c6a-impurity: gini,
	dtc_53326fbc1c6a-maxDepth: 4,
	dtc_53326fbc1c6a-minInstancesPerNode: 2
}, {
	dtc_53326fbc1c6a-impurity: entropy,
	dtc_53326fbc1c6a-maxDepth: 5,
	dtc_53326fbc1c6a-minInstancesPerNode: 2
}, {
	dtc_53326fbc1c6a-impurity: gini,
	dtc_53326fbc1c6a-maxDepth: 5,
	dtc_53326fbc1c6a-minInstancesPerNode: 2
}, {
	dtc_53326fbc1c6a-impurity: entropy,
	dtc_53326fbc1c6a-maxDepth: 6,
	dtc_53326fbc1c6a-minInstancesPerNode: 2
}, {
	dtc_53326fbc1c6a-impurity: gini,
	dtc_53326fbc1c6a-maxDepth: 6,
	dtc_53326fbc1c6a-minInstancesPerNode: 2
}, {
	dtc_53326fbc1c6a-impurity: entropy,
	dtc_53326fbc1c6a-maxDepth: 7,
	dtc_53326fbc1c6a-minInstancesPerNode: 2
}, {
	dtc_53326fbc1c6a-impurity: gini,
	dtc_53326fbc1c6a-maxDepth: 7,
	dtc_53326fbc1c6a-minInstancesPerNode: 2
}, {
	dtc_53326fbc1c6a-impurity: entropy,
	dtc_53326fbc1c6a-maxDepth: 8,
	dtc_53326fbc1c6a-minInstancesPerNode: 2
}, {
	dtc_53326fbc1c6a-impurity: gini,
	dtc_53326fbc1c6a-maxDepth: 8,
	dtc_53326fbc1c6a-minInstancesPerNode: 2
}, {
	dtc_53326fbc1c6a-impurity: entropy,
	dtc_53326fbc1c6a-maxDepth: 9,
	dtc_53326fbc1c6a-minInstancesPerNode: 2
}, {
	dtc_53326fbc1c6a-impurity: gini,
	dtc_53326fbc1c6

## 3. Random Forest Classification Model

In [0]:
%scala

// Initiating the model
val rfModel = new RandomForestClassifier().setLabelCol("label").setFeaturesCol("features").setNumTrees(50)

// Defining the hyper-parameter grid
val paramGrid = new ParamGridBuilder()
    .addGrid(rfModel.featureSubsetStrategy, Array("auto", "all", "sqrt"))
    .addGrid(rfModel.minInstancesPerNode, Array(1, 2, 3))
    .build()

// Evaluating AUC 
val evaluator = new BinaryClassificationEvaluator()

// Performing cross-validation for model selection
val cv = new CrossValidator().setEstimator(rfModel).setEstimatorParamMaps(paramGrid).setEvaluator(evaluator).setNumFolds(5)

// Training the model
val cvModel = cv.fit(trainSet)

// Making predictions on test data
val predictions = cvModel.transform(testSet)

print("The best random forest model has the following attributes:\n")
print(cvModel.bestModel.extractParamMap())

print("\nThe AUC of the best random forest model is: ")
print(evaluator.evaluate(predictions))

// Computing other metrics for the classifier
compute_metrics(predictions, "Random Forest")

The best random forest model has the following attributes:
{
	rfc_7b0b871b0756-bootstrap: true,
	rfc_7b0b871b0756-cacheNodeIds: false,
	rfc_7b0b871b0756-checkpointInterval: 10,
	rfc_7b0b871b0756-featureSubsetStrategy: auto,
	rfc_7b0b871b0756-featuresCol: features,
	rfc_7b0b871b0756-impurity: gini,
	rfc_7b0b871b0756-labelCol: label,
	rfc_7b0b871b0756-leafCol: ,
	rfc_7b0b871b0756-maxBins: 32,
	rfc_7b0b871b0756-maxDepth: 5,
	rfc_7b0b871b0756-maxMemoryInMB: 256,
	rfc_7b0b871b0756-minInfoGain: 0.0,
	rfc_7b0b871b0756-minInstancesPerNode: 1,
	rfc_7b0b871b0756-minWeightFractionPerNode: 0.0,
	rfc_7b0b871b0756-numTrees: 50,
	rfc_7b0b871b0756-predictionCol: prediction,
	rfc_7b0b871b0756-probabilityCol: probability,
	rfc_7b0b871b0756-rawPredictionCol: rawPrediction,
	rfc_7b0b871b0756-seed: 207336481,
	rfc_7b0b871b0756-subsamplingRate: 1.0
}
The AUC of the best random forest model is: 0.7567443382538994
For the Random Forest classifier, the following evaluation metrics were computed: 
The accuracy is: 0.8025337837837838
The precision is: 0.6789587852494577
The recall is: 0.23463268365817092
The F1 score is: 0.3487465181058496rfModel: org.apache.spark.ml.classification.RandomForestClassifier = rfc_7b0b871b0756
paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	rfc_7b0b871b0756-featureSubsetStrategy: auto,
	rfc_7b0b871b0756-minInstancesPerNode: 1
}, {
	rfc_7b0b871b0756-featureSubsetStrategy: auto,
	rfc_7b0b871b0756-minInstancesPerNode: 2
}, {
	rfc_7b0b871b0756-featureSubsetStrategy: auto,
	rfc_7b0b871b0756-minInstancesPerNode: 3
}, {
	rfc_7b0b871b0756-featureSubsetStrategy: all,
	rfc_7b0b871b0756-minInstancesPerNode: 1
}, {
	rfc_7b0b871b0756-featureSubsetStrategy: all,
	rfc_7b0b871b0756-minInstancesPerNode: 2
}, {
	rfc_7b0b871b0756-featureSubsetStrategy: all,
	rfc_7b0b871b0756-minInstancesPerNode: 3
}, {
	rfc_7b0b871b0756-featureSubsetStrategy: sqrt,
	rfc_7b0b871b0756-minInstancesPerNode: 1
}, {
	rfc_7b0b871b0756-featureSubsetStrategy: sqrt,
	rfc_7b0b871b0756-minInstancesPerNode: 2
}, {
	rfc_7b0b871b0756-featureSubsetStrategy: sqrt,
	rfc_7b0b871b0756-minInstancesPerNode: 3
})
evaluator: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = BinaryClassificationEvaluator: uid=binEval_7a3f64aa2494, metricName=areaUnderROC, numBins=1000
cv: org.apache.spark.ml.tuning.CrossValidator = cv_c57639a09440
cvModel: org.apache.spark.ml.tuning.CrossValidatorModel = CrossValidatorModel: uid=cv_c57639a09440, bestModel=RandomForestClassificationModel: uid=rfc_7b0b871b0756, numTrees=50, numClasses=2, numFeatures=91, numFolds=5
predictions: org.apache.spark.sql.DataFrame = [features: vector, label: int ... 3 more fields]

Summary of the metrics for each classification model:

AUC:
* Logistic Regression (LR): 0.766
* Decision Tree (DT): 0.479
* Random Forest (RF): 0.756

Other Metrics:

      LR               DT               RF   
    0.8149           0.8073           0.8025      - Accuracy
    0.6705           0.6447           0.6789      - Precision
    0.3508           0.3223           0.2346      - Recall
    0.4606           0.4298           0.3487      - F1 Score

## Conclusion

Based on the results from all three classification models,

With regards to AUC, the scores of Logistic Regression and Random Forest are very similar and they outperform Decision Tree. But, as always, Logistic Regressions seems to be the best model for binary classification since it has a slight edge over RF with a slightly high measure of separability.

But, with regards to other metrics such as accuracy, precision, recall and F1 score, we can see that the decision tree is good at distinguishing the different classes. Taking this into consider, it is evident that the decision tree classifier doesn't underperform in comparison to the logistic regression and random forest classifiers, which was a conclusion that we derived by considering the AUC values. For some reason, we do not get a good AUC value in Spark is what we think.

Having seen all metrics, F1 Score seems to be a better metric to take into consideration since there is a class imbalance. But, if in case when we have the same amount of data points in every class, then accuracy will be a better metric.

To conclude, all the three classifiers performed really good but it took a long time in Spark to get computed. We still feel the AUC value for decision tree classifier will be a lot better if not computed in Spark but overall we got a good overview and understanding of how well the classification models work.